# Sentence to PClean query demo
This notebook demos the sentence-to-PClean pipeline. Given a sentence giving information about some doctor, it generates and runs queries about those people against the Medicare dataset used in Alex Lew's paper using a PClean model trained by Ian Limarta.


In [1]:
import gc
import json
import logging
from pathlib import Path
import sqlite3
import string
from typing import Any, Optional, TypeVar, Union

from IPython.display import display, Markdown
import lark
import nest_asyncio
import requests
import spacy
import torch
import transformers
from tqdm import tqdm
import vllm

import genparse
from genparse import InferenceSetup, InferenceSetupVLLM

nest_asyncio.apply()

logger = logging.getLogger("notebook")

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(name)s -   %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S',
)

In [2]:
debug_sentences_path = Path().resolve() / "debug_sentences.jsonl"
debug_sentences = [json.loads(line) for line in debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

In [3]:
detailed_debug_sentences_path = Path().resolve() / "debug_sentences_detailed.jsonl"
detailed_debug_sentences = [json.loads(line) for line in detailed_debug_sentences_path.read_text(encoding="utf-8").splitlines() if line.strip()]

## Example inputs and outputs

Taken from the first ten sentences (rows 2-11) in the [GPT-4 tweets Marjorie Freedman generated][gpt4_tweets].

[gpt4_tweets]: https://docs.google.com/spreadsheets/d/1vq_HAdbFY079vYVOppkFLk_zojJ46MJKsy-TzUm7Q3M/edit

In [7]:
try:
    sentence_table_row_template = string.Template("| $sentence | <pre>$example_output</pre> |")
    def show_sentences(sentence_data):
        lines = ["| Sentence | Example PClean preamble code |", "| --- | --- |"]
        for sentence_datum in sentence_data:
            lines.append(sentence_table_row_template.substitute(sentence=sentence_datum["sentence"], example_output=sentence_datum["example_output"].replace("\n", "<br>")))
        markdown = "\n".join(lines)
        display(Markdown(markdown))
    
    
    show_sentences(debug_sentences)
except KeyError as e:
    display(Markdown(f'Oops, one of these examples is missing a key... `{e}`'))

Oops, one of these examples is missing a key... `'example_output'`

## Helper code

### SpaCy code

In [8]:
# Supporting code for entity extraction.
_SPACY_MODEL_NAME = 'en_core_web_trf'
spacy_model = spacy.load(_SPACY_MODEL_NAME)
_PERSON_LABEL = 'PERSON'
_LOCATION_LABEL = 'LOC'
# spaCy labels list example thanks to Stack Overflow user 'russhoppa': https://stackoverflow.com/a/78252807
_SPACY_LABELS = spacy_model.get_pipe("ner").labels
assert _PERSON_LABEL in _SPACY_LABELS

T = TypeVar('T')

def _uniquify(items: list[T]) -> list[T]:
    """
    O(n^2) order-preserving uniquification.

    Fine for short lists like a single sentence's list of PERSON entities.
    """
    result = []
    for item in items:
        if item not in result:
            result.append(item)
    return result

def get_people(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _PERSON_LABEL]
    )

def get_locations(sentence: str) -> list[str]:
    return _uniquify(
        [ent.text for ent in spacy_model(sentence).ents if ent.label_ == _LOCATION_LABEL]
    )

def show_ents(sentence: str) -> None:
    print(sentence)
    print(spacy_model(sentence).ents)

for sentence_datum in debug_sentences:
    show_ents(sentence_datum["sentence"])
    print()

Just had an insightful consultation with Dr. Kay Ryan in Baltimore. Feeling optimistic about my health! #HealthMatters #Baltimore
(Kay Ryan, Baltimore, Baltimore)

Dr. Kay Ryan's office at 321 Pine St. in Baltimore is so welcoming and efficient. Highly recommend! #CityCare
(Kay Ryan, Baltimore, CityCare)

Exploring Baltimore after an informative appointment with Dr. Kay Ryan. Loving this city's vibe! #BaltimoreAdventures #DoctorVisit
(Baltimore, Kay Ryan)

Dr. Kay Ryan, a neurologist, provided excellent care today. Grateful for her expertise! #Healthcare #BaltimoreDoctors
(Kay Ryan, today)

Feeling reassured after my visit to Dr. Kay Ryan in Baltimore. She's truly exceptional! #FeelingGood #CityOfCharm
(Kay Ryan, Baltimore)

Had a great experience with Dr. Kay Ryan at 321 Pine St. in Baltimore. Her attention to detail is unmatched! #HealthCheck
(Kay Ryan, Baltimore)

From diagnosis to treatment, Dr. Kay Ryan, a cardiologist, covered it all. Baltimore, you're lucky to have her! #Medical

### Extracting the generated code from a response

In [9]:
def extract_code_from_response(text: str) -> str:
    result = text.removeprefix('<|start_header_id|>assistant<|end_header_id|>').strip()
    return result

### Sort posterior

In [10]:
def sort_posterior(posterior):
    return {inference: likelihood for inference, likelihood in sorted(posterior.items(), key=lambda t: (t[1], t[0]), reverse=True)}

### Aggregate likelihoods over extracted code

In [11]:
def get_aggregate_likelihoods(posterior: dict[str, float]) -> dict[str, float]:
    result = {}
    for inference, likelihood in posterior.items():
        code_only = extract_code_from_response(inference)
        try:
            code_only = json.dumps(
                {key: value for key, value in sorted(json.loads(code_only).items(), key=lambda t: t[0])},
                indent=2
            )
        except json.JSONDecodeError:
            pass
        result.setdefault(code_only, 0.0)
        result[code_only] += likelihood
    return sort_posterior(result)

### Get best inference

In [12]:
def get_best_inference(posterior: dict[str, float]) -> tuple[str, float]:
    return max(posterior.items(), key=lambda t: (t[1], t[0]))

### Running inference locally

In [13]:
SERVER_MODEL_ID = 'meta-llama/Meta-Llama-3.1-8B-Instruct'
SERVER_GENPARSE_MODEL_NAME = "llama3.1"
_LOCAL_BATCH_SIZE = 1

class FakeResponse:
    def __init__(self, data):
        self._data = data
    
    @property
    def status_code(self):
        return 200

    def json(self):
        return self._data

    def text(self):
        return json.dumps(self._data)

def server_model(grammar, proposal="character"):
    return InferenceSetupVLLM(SERVER_GENPARSE_MODEL_NAME, grammar, proposal_name=proposal)

def run_inference_vllm(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _LOCAL_BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = "",
    inference_setup: InferenceSetupVLLM,
) -> requests.Response:
    #assert batch_size == inference_setup.sampler.llm._model.batch_size
    assert grammar == inference_setup.sampler.grammar
    assert (proposal == 'character' and isinstance(inference_setup, genparse.CharacterProposal))
    inference = inference_setup(prompt, n_particles=n_particles, temperature=temperature, max_tokens=max_tokens)
    return FakeResponse({"log_ml": None, "posterior": inference.posterior})

### Running the server

In [14]:
_BATCH_SIZE = 1
# Ben LeBrun's WIP server running on GCP as of 2024-07-16
_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.122.30.137:9999/restart'
_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.122.30.137:8888/infer'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI = 'http://34.70.201.1:9999/restart'
#_DEFAULT_GENPARSE_INFERENCE_SERVER_URI = 'http://34.70.201.1:8888/infer'

def restart_inference_server():
    basic = requests.auth.HTTPBasicAuth(input("User: "), input('Password: '))
    result = requests.post(_DEFAULT_GENPARSE_INFERENCE_SERVER_RESTART_URI, auth=basic)
    return result

def run_inference_server(
    prompt: str,
    *,
    proposal: str = 'character',
    batch_size: int = _BATCH_SIZE,
    max_tokens: int,
    n_particles: int,
    temperature: float = 1.0,
    grammar: str,
    genparse_url: str = _DEFAULT_GENPARSE_INFERENCE_SERVER_URI,
) -> requests.Response:
    """
    Run inference using a server.
    """
    params = {
        'prompt': prompt,
        'method': 'smc-standard',
        'n_particles': n_particles,
        'lark_grammar': grammar,
        'proposal_name': proposal,
        'proposal_args': {},
        'max_tokens': max_tokens,
        'temperature': temperature,
    }
    headers = {
        "Content-type": "application/json",
        "Accept": "application/json"
    }
    response = requests.post(genparse_url, json=params, headers=headers)

    return response

In [15]:
#restart_inference_server()

### JSON generation prompt

In [ ]:
json_prompt = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    “some_feature”: “foo”,
    “another_feature”: “bar”,
    ...
    “last_feature”: “baz”
}
```

There are eight features we would like to extract from the sentence:

- “first” (the doctor’s given name)
- “last” (the doctor’s surname)
- “specialty” (the doctor’s specialty)
- “legal_name” (the legal name of the doctor’s business, practice, or employer)
- “city_name” (the city where the doctor’s office is)
- “zip” (the ZIP code of the doctor’s office)
- “addr” (the doctor’s office address, “line 1" -- this is the street address)
- “addr2” (additional address information giving a suite number, unit number, et cetera -- “line 2" of the address)

Please generate a key-value pair for every such value specified in the sentence. Avoid nulls and empty strings. Omit any missing values. If the sentence calls the doctor “Dr. John Smith,” include both the given and surname (`“first”` and `“last”`). If the sentence calls the doctor “Dr. Smith,” include only the surname (`“last”`). Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: What a great experience with Dr. Williams’s pediatrics office in Victoria! My child has never been happier to be jabbed with a needle. #ParentingVictories #Victoria
Output: ```json
{“last”: “Williams”, “specialty”: “pediatrics”, “city_name”: “Victoria”}
```

Input: John Smith’s neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{“first”: “John”, “last”: “Smith”, “specialty”: “neurology”, “legal_name”: “Happy Brain Services LLC”, “addr”: “512 Example Street”, “addr2": “Suite 3600”, “city_name”: “Camp Hill”}
```

Input: Loved visiting Dr. Kay Ryan’s neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{“first”: “Kay”, “last”: “Ryan”, “legal_name”: “Green Medicine Inc”, “specialty”: “neurology”, “addr”: “256 Overflow St”, “zip”: “170112202”, “city_name”: “Camp Hill”}
```

Input: Dr. Pat Rogers’s office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{“first”: “Pat”, “last”: “Rogers”, “legal_name”: “Soulful Medical Services Inc”, “city_name”: “Baltimore”}
```

Input: Dr. Maynard’s really rules! They figured out my illness when no one else could. Loving the care here in #Lexington
Output: ```json
{“last”: “Maynard”, “city_name”: “Lexington”}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt(sentence: str) -> str:
    return json_prompt.substitute(sentence=sentence)

print(json_prompt)

In [ ]:
json_prompt2 = string.Template(
    """Write a flat JSON object describing one of the doctors in the given input sentence.

In general, your output should look like:

```json
{
    "some_feature": "foo",
    "another_feature": "bar",
    ...
    "last_feature": "baz"
}
```

There are eight features we would like to extract from the sentence:

- "first" (the doctor's given name)
- "last" (the doctor's surname)
- "specialty" (the doctor's specialty)
- "legal_name" (the legal name of the doctor's business, practice, or employer)
- "city_name" (the city where the doctor's office is)
- "zip" (the ZIP code of the doctor's office)
- "addr" (the doctor's office address, "line 1" -- this is the street address)
- "addr2" (additional address information giving a suite number, unit number, et cetera -- "line 2" of the address)

Please generate for each feature a key-value pair corresponding to the information given in the sentence. If the sentence doesn't specify a feature, simply output `null`. If the sentence calls the doctor "Dr. John Smith," include both the given name (`"first"`) and the surname (`"last"`). If the sentence calls the doctor "Dr. Smith," assume it is the surname and include only that surname (as `"last"`). Do not write a script. Output the JSON itself directly with no preamble or commentary. The following examples illustrate how you should behave on the input sentence.

Input: What a great experience with Dr. Williams's pediatrics office in Victoria! My child has never been happier to be jabbed with a needle. #ParentingVictories #Victoria
Output: ```json
{"first": null, "last": "Williams", "specialty": "pediatrics", "legal_name": null, "city_name": "Victoria", "zip": null, "addr": null, "addr2": null}
```

Input: John Smith's neurology office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (Camp Hill) is terrible!
Output: ```json
{"first": "John", "last": "Smith", "specialty": "neurology", "legal_name": "Happy Brain Services LLC", "city_name": "Camp Hill", "zip": null, "addr": "512 Example Street", "addr2": "Suite 3600"}
```

Input: Loved visiting Dr. Kay Ryan's neurology office (Green Medicine Inc.) at 256 Overflow St (ZIP 17011-2202)! No wait time at all. #CampHill
Output: ```json
{"first": "Kay", "last": "Ryan", "specialty": "neurology", "legal_name": "Green Medicine Inc", "city_name": "Camp Hill", "zip": "170112202", "addr": "256 Overflow St", "addr2": null}
```

Input: Dr. Pat Rogers's office screwed us! So much for Soulful Medical Services Inc. Took our money and Pat gave us three minutes tops. #BaltimoreSucks
Output: ```json
{"first": "Pat", "last": "Rogers", "specialty": null, "legal_name": "Soulful Medical Services Inc", "city_name": "Baltimore", "zip": null, "addr": null, "addr2": null}
```

Input: Dr. Maynard's really rules! They figured out my illness when no one else could. Loving the care here in #Lexington
Output: ```json
{"first": null, "last": "Maynard", "specialty": null, "legal_name": null, "city_name": "Lexington", "zip": null, "addr": null, "addr2": null}
```

The following is your input sentence. Produce the appropriate output.

Input: $sentence
Output:"""
)
# scrapped
# The doctor's name is $name. Their office address is $address. Their specialty is $specialty. The city is $city.

def format_json_prompt2(sentence: str) -> str:
    return json_prompt2.substitute(sentence=sentence)

print(json_prompt2)

### JSON grammar

In [ ]:
json_grammar = r"""
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair ("," NL? OPTIONAL_WS kv_pair) ~ 0..7 NL? OPTIONAL_WS "}"
kv_pair: COLUMN OPTIONAL_WS ":" OPTIONAL_WS value
value: STRING | NULL

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\\n"
NULL: "null"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
COLUMN: "\"" ("first" | "last" | "addr" | "addr2" | "specialty" | "city_name" | "zip" | "legal_name") "\""
"""

In [ ]:
json_grammar2 = """
start: _form_no_fence
_form_no_fence: _ASSISTANT_TAG NL NL? NL? json
_ASSISTANT_TAG: "<|start_header_id|>assistant<|end_header_id|>"
json: "{" NL? OPTIONAL_WS kv_pair{FIRST, value{/"[a-zA-Z -]{0,20}"/}} NL? OPTIONAL_WS kv_pair{LAST, value{/[a-zA-Z -]{0,20}/}} NL? OPTIONAL_WS kv_pair{SPECIALTY, value{/"[a-zA-Z]{0,20}"/}} NL? OPTIONAL_WS kv_pair{LEGAL_NAME, value{/"[a-zA-Z &.-]{0,50}"/}} NL? OPTIONAL_WS kv_pair{CITY_NAME, value{/"[a-zA-Z -]{0,30}/}} NL? OPTIONAL_WS kv_pair{ZIP, value{/|([0-9]{5,9})/}} NL? OPTIONAL_WS kv_pair{ADDR, value{/"[a-zA-Z0-9 &.-]{0,50}"/}} NL? OPTIONAL_WS kv_pair{ADDR2, value{/"[a-zA-Z0-9 .-]{0,20}"/}} NL? OPTIONAL_WS "}"
kv_pair{col_, val}: col_ OPTIONAL_WS ":" OPTIONAL_WS val
value{something}: something | NULL

CODE_FENCE: "```"
JSON_TAG: "json"
OPTIONAL_WS: WS? WS? WS? WS?
WS: " "
NL: "\\n"
NULL: "null"
STRING: /"[a-zA-Z0-9-.,'& ]{0,60}"/
col{name}: "\\"" name "\\""
FIRST: col{"first"}
LAST: col{"last"}
SPECIALTY: col{"specialty"}
LEGAL_NAME: col{"legal_name"}
CITY_NAME: col{"city_name"}
ZIP: col{"zip"}
ADDR: col{"addr"}
ADDR2: col{"addr2"}
"""

## Debugging

In [ ]:
model_id = 'meta-llama/Meta-Llama-3.1-8B-Instruct'

In [ ]:
def md_html_output_text_for_json(text: str) -> str:
    return text.replace("<", "&lt;").replace(">", "&gt;").replace("|", "\|").replace("\n", "<br>").replace("```", "\\```")

### Running the LM unconstrained

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

In [ ]:
# Taken from a magic number in the Genparse VLLM backend code.
_MAX_MODEL_LEN = 4096
# Why dtype=torch.float32? It's in the Genparse VLLM backend code.
model = vllm.LLM(model_id, dtype=torch.float32, max_model_len=_MAX_MODEL_LEN)

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
sentence = debug_sentences[-1]["sentence"]
print(sentence)
people = get_people(sentence)
prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt(sentence=sentence)}], tokenize=False)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')

sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
response = model.generate(prompt, sampling_params=sampling_params)[0]
for i, output in enumerate(response.outputs, start=1):
    print(f'Generated Query {i}: ```{output.text}```')

In [ ]:
sentence_to_outputs = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    print(sentence)
    prompt = tokenizer.apply_chat_template([{'role': 'user', 'content': format_json_prompt2(sentence=sentence)}], tokenize=False)
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)
    
    sampling_params = vllm.SamplingParams(temperature=1.0, max_tokens=128, n=25)
    response = model.generate(prompt, sampling_params=sampling_params)[0]
    sentence_to_outputs[sentence] = []
    for i, output in enumerate(response.outputs, start=1):
        sentence_to_outputs[sentence].append(output)
        print(f'Generated Query {i}: ```{output.text}```')

In [ ]:
table_rows = ['| Sentence | Extracted Code | Raw Response | # Tokens | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:| ---:|']
for sentence, outputs in sentence_to_outputs.items():
    try:
        posterior = {}
        n_outputs = {}
        total_tokens = {}
        for i, output in enumerate(outputs):
            posterior.setdefault(output.text, 0.0)
            posterior[output.text] += 1.0 / len(outputs)
            n_outputs.setdefault(output.text, 0)
            n_outputs[output.text] += 1
            total_tokens.setdefault(output.text, 0)
            total_tokens[output.text] += len(output.token_ids)
        aggregate_likelihoods = sort_posterior(get_aggregate_likelihoods(posterior))
        n_outputs_aggregated = {}
        total_tokens_aggregated = {}
        for output in outputs:
            code = extract_code_from_response(output.text)
            try:
                code = json.dumps({key: value for key, value in sorted(json.loads(code).items())}, indent=2)
            except json.JSONDecodeError:
                pass
            n_outputs_aggregated.setdefault(code, 0)
            n_outputs_aggregated[code] += n_outputs[output.text]
            total_tokens_aggregated.setdefault(code, 0)
            total_tokens_aggregated[code] += total_tokens[output.text]
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| {total_tokens_aggregated[code] / n_outputs_aggregated[code]}'
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError as e:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {e} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

In [ ]:
try:
    del model
    gc.collect()
except NameError:
    pass

### Running the model with constraints

In [ ]:
try:
    tokenizer
except NameError:
    tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)

In [ ]:
sentence = debug_sentences[-1]["sentence"]
prompt = tokenizer.apply_chat_template(
    [
        {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
    ],
    tokenize=False,
)
print(f'Prompt ({len(tokenizer(prompt)["input_ids"])} tokens): ```{prompt}```')


# ignored top_p=0.95
server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar2}
response = run_inference_server(prompt, **server_inference_params)
try:
    data = response.json()
    for i, (query, likelihood) in enumerate(data['posterior'].items(), start=1):
        
        print(f'Generated Query {i} (likelihood {100 * likelihood:.2f}%): ```{query}```')
except json.JSONDecodeError:
    print(response.status_code)
    print(response.text)

In [ ]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    print(response)

In [ ]:
table_rows = ['| Sentence | Extracted Code | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        aggregate_likelihoods = get_aggregate_likelihoods(posterior)
        # for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
        #     # Assert the response parses properly
        #     pclean_parser.parse(output_text)

        #     code = extract_code_from_response(output_text)
        #     aggregate_likelihood = aggregate_likelihoods[code]
        #     code_with_br = code.replace("\n", "<br>")
        #     output_with_br = md_html_output_text(output_text)
        #     table_rows.append(
        #         f'| {sentence if i == 1 else "   "} '
        #         f'| <pre>{code_with_br}</pre> '
        #         f'| <pre>{output_with_br}</pre> '
        #         f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
        #     )
        for i, (code, likelihood) in enumerate(aggregate_likelihoods.items(), start=1):
            aggregate_likelihood = aggregate_likelihoods[code]
            code_with_br = code.replace("\n", "<br>")
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre>{code_with_br}</pre> '
                f'| (unk) '
                f'| {100 * aggregate_likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))

### Local inference

In [ ]:
inference_setup = server_model(json_grammar)

In [ ]:
print(inference_setup.sampler.llm._model.batch_size)

In [ ]:
dir(inference_setup.sampler.llm._model)

In [ ]:
responses = {}
for sentence_datum in debug_sentences:
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 128, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_vllm(prompt, inference_setup=inference_setup, **server_inference_params)
    responses[sentence] = response

In [ ]:
try:
    del inference_setup
except NameError:
    pass

### Running the model on an arbitrary sentence

In [ ]:
responses = {}
for sentence_datum in tqdm([#{"sentence": "John Smith the podiatrist is terrible, so much worse than John Smith the cardiologist."}, {"sentence": "John Smith works with Jack Smith at the neurology office. He’s terrible at neurology."},
                            {"sentence": "John Smith's office (Happy Brain Services LLC) at 512 Example Street Suite 3600 (CA-170) is terrible!"}]):
    sentence = sentence_datum["sentence"]
    prompt = tokenizer.apply_chat_template(
        [
            {'role': 'user', 'content': format_json_prompt(sentence=sentence)}
        ],
        tokenize=False,
    )
    logger.debug('Prompt (%d tokens): ```%s```', len(tokenizer(prompt)["input_ids"]), prompt)

    # ignored top_p=0.95
    server_inference_params = {'max_tokens': 256, 'n_particles': 15, 'temperature': 1.0, 'grammar': json_grammar}
    response = run_inference_server(prompt, **server_inference_params)
    responses[sentence] = response
    #break

In [ ]:
table_rows = ['| Sentence | Extracted JSON | Raw Response | Aggregate Likelihood (Raw LH) |', '|:--- |:--- |:--- | ---:|']
for sentence, response in responses.items():
    try:
        data = response.json()
        posterior = data['posterior']
        for i, (output_text, likelihood) in enumerate(posterior.items(), start=1):
            # Assert the response parses properly
            # TODO grammar no longer parses in lark but does in genparse so idk, can't test this as easily
            #json_parser.parse(output_text)

            #code = extract_code_from_response(output_text)
            code = output_text
            code_with_br = code.replace("\n", "<br>")
            output_with_br = md_html_output_text_for_json(output_text)
            table_rows.append(
                f'| {sentence if i == 1 else "   "} '
                f'| <pre></pre> '
                f'| <pre>{output_with_br}</pre> '
                f'| {100 * likelihood:.2f}% ({100 * likelihood:.2f}%) |'
            )
    except json.JSONDecodeError:
        table_rows.append(
            f'| {sentence if i == 1 else "   "} | ERROR: {response.status_code} - {response.text} | 100% |'
        )
display(Markdown('\n'.join(table_rows)))